In [0]:
# import pytroch geometric hetero data
# from torch_geometric.data import HeteroData
# import torch
# data = HeteroData.from_dict(torch.load('HeteroData_Learnings_normalized_triangles_withadditionaldata_v1.pt'))

In [0]:
import os

if "DATABRICKS_RUNTIME_VERSION" in os.environ and not 'installed_libs' in globals():
  #CUDA = 'cu121' 
  installed_libs = True
  
  
  !pip install torch==2.1.0  torchvision==0.16.0 torchtext==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121
  import torch
  #os.environ['TORCH'] = torch.__version__
  #print(torch.__version__)
  #torch_version = '2.0.0+cu118'
  
  #!pip install pyg_lib torch_scatter torch_sparse torch_cluster -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html # torch_spline_conv
  !pip install torch_geometric
  !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html
  #!pip install torch_sparse -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
  #!pip install torch_scatter -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
  #!pip install pyg_lib -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
  !pip install sentence-transformers
  !pip install torcheval
  !pip install matplotlib
  !pip install pandas
  !pip install tensorboard
  
if "DATABRICKS_RUNTIME_VERSION" in os.environ:
  ROOT_FOLDER = '/dbfs/FileStore/GraphNeuralNetworks/'
else:
  ROOT_FOLDER = ''

In [0]:
from learnings_sampler_v1 import get_datasets

In [0]:
train_data, val_data, test_data = get_datasets(get_edge_attr=False,filename=ROOT_FOLDER+'HeteroData_Learnings_normalized_triangles_withadditionaldata_v1.pt', filter_top_k=True, top_k=50)

In [0]:
# people have jobs, if multiple, we average
# jobs have skills, if multiple, we average
#  

# so row is [pid, lid][person learning person_features person_jobs job_skills target_watched]

In [0]:
import torch
from tqdm.auto import tqdm
from torch_geometric.data import HeteroData
import os 

def make_ds(data, split_name):
    def skills_avg(edge_index_to_skill, entity_id):
        # eg job->skill, entity_id = job 254
        mask1 = edge_index_to_skill[0,:]==entity_id
        mask2 = edge_index_to_skill[1,:][mask1].squeeze()
        if torch.sum(mask1)==0:
            return torch.zeros(1, data['skills'].x.shape[1])
        a = data['skills'].x[mask2]
        if a.dim() == 1:
            return a.unsqueeze(0)
        else:
            return a.mean(axis=0).squeeze().unsqueeze(0)

    
    job_skills = []
    for job_id in tqdm(range(data['jobs'].num_nodes)):
        e= skills_avg(data['jobs', 'rev_job_skill', 'skills'].edge_index,job_id)
        job = data['jobs'].x[job_id,:].unsqueeze(0)
        
        job_skills.append(torch.cat((job,e),dim=1))
    
    learning_skills = []
    for learning_id in tqdm(range(data['courses_and_programs'].num_nodes)):
        e  = skills_avg(data['courses_and_programs', 'rev_course_and_program_skill', 'skills'].edge_index,learning_id)
        learning = data['courses_and_programs'].x[learning_id,:].unsqueeze(0)
        learning_skills.append(torch.cat((learning,e),dim=1))
      
        
        
    jobs_skills = torch.cat((job_skills),dim=0)
    
    torch.save(torch.cat((learning_skills),dim=0),ROOT_FOLDER+'learnings/'+split_name+'/learnings.pt')
    del learning_skills

    def avg_of_computed(edge_index_to_skill, opposite_entity_id, opposite_datax):
        # eg job->skill, oposite_entity_id = job 254
        mask1 = edge_index_to_skill[0,:]==opposite_entity_id
        mask2 = edge_index_to_skill[1,:][mask1].squeeze()
        if torch.sum(mask1)==0:
            return torch.zeros(1, opposite_datax.shape[1])
        a = opposite_datax[mask2]
        if a.dim() == 1:
            return a.unsqueeze(0)
        else:
            return a.mean(axis=0).squeeze().unsqueeze(0)
    
    def avg_onehot_of_computed(edge_index_to_skill, opposite_entity_id, opposite_name):
        # eg job->skill, oposite_entity_id = job 254
        mask1 = edge_index_to_skill[0,:]==opposite_entity_id
        mask2 = edge_index_to_skill[1,:][mask1].squeeze()
        #onehot = torch.zeros(1,data[opposite_name].num_nodes)
        if mask2.numel()==0:
            return torch.tensor([[-1]])
        elif mask2.dim()==0:
            return torch.tensor([[mask2.item()]])
        else:
            return torch.tensor([[mask2[0].item()]])
        if mask2.dim()==0:
            onehot[0,mask2.item()] = 1
        
        else:
            for i in mask2:
                onehot[0,i] = 1

        return onehot
    
    

    persons = []


    

    
    for person_i in tqdm(range(data['people'].x.shape[0])):
        pid = person_i 
        supervisor = avg_onehot_of_computed(data['people','supervisor_supervisee','people'].edge_index,person_i,'people')
        organization = avg_onehot_of_computed(data['people', 'organization_student', 'organizations'].edge_index,person_i,'organizations')

        jobs = avg_of_computed(data['people','rev_job_student','jobs'].edge_index, person_i, jobs_skills)
        person = data['people'].x[person_i,:].unsqueeze(0)

        persons.append(torch.cat((person,jobs,supervisor,organization),dim=1)) # 
    
    persons = torch.cat((persons),dim=0)
    
    dataset = HeteroData()
    dataset['people'].x = persons
  
    dataset['courses_and_programs'].x =  torch.load(ROOT_FOLDER+'learnings/'+split_name+'/learnings.pt')
    dataset['people', 'completed','courses_and_programs']['edge_index'] = data['people', 'rev_course_and_programs_student', 'courses_and_programs'].edge_index
    a = data['courses_and_programs','course_and_programs_student','people'].edge_label_index
    
    dataset['people', 'completed','courses_and_programs']['edge_label_index'] = torch.cat((a[1,:].unsqueeze(0),a[0,:].unsqueeze(0)),dim=0)
    
    



    filename = ROOT_FOLDER+f'FactorizationMachines_Dataset_{split_name}_v1.pt'
    if os.path.exists(filename):
        raise Exception('File already exists')
    else:
        torch.save(dataset.to_dict(), filename)

In [0]:
#make_ds(train_data,'train')
make_ds(val_data, 'val')
make_ds(test_data, 'test') 

In [0]:
# sampler

def nf_sampler(batch_size, neg_sample_ratio=1, edge_label_index, total_num_target_nodes, num_organizations):
    # triplet mode only
    # sample some random edges
    num_samples = batch_size
    sampled_indices = torch.randint(0, edge_label_index.shape[1], (num_samples,))
    sampled_edges = edge_label_index[:, sampled_indices]

    neg_samples = batch_size*neg_sample_ratio
    
    s = sampled_edges[0,:].unsqueeze(0)
    src_edges = s
    for i in range(neg_sample_ratio-1):
        src_edges= torch.cat((src_edges,s),dim=1)

    sampled_negatives = torch.randint(0, num_organizations, (neg_samples,)).squeeze().unsqueeze(0)
   

    negative_edge_label_index = torch.cat((src_edges, sampled_negatives),dim=0)
    edge_label_indices = torch.cat((sampled_edges, negative_edge_label_index),dim=1)
    
    return edge_label_indices, torch.cat((torch.ones(batch_size), torch.zeros(neg_samples)))
    
    
    